In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import nltk

In [ ]:
df = pd.read_csv("spam.csv")

In [ ]:
df.info()

## 1. Data Cleaning

In [ ]:
# xóa cột không cần thiết
df.drop(columns=["Unnamed: 2", "Unnamed: 3", "Unnamed: 4", "Unnamed: 5", 
                 "Unnamed: 6", "Unnamed: 7", "Unnamed: 8", "Unnamed: 9", 
                 "Unnamed: 10", "Unnamed: 11", "Unnamed: 12", "Unnamed: 13", 
                 "Unnamed: 14", "Unnamed: 15", "Unnamed: 16"], inplace=True)

### Handling the missing value

In [ ]:
df.isnull().sum()

In [ ]:
no_label = df # tập dữ liệu không có label thỏa mãn nhưng có text

In [ ]:
no_label.dropna(subset=["text"], inplace=True)

In [ ]:
valid = ["spam", "ham"]
no_label = no_label[~no_label["label"].isin(valid)]

In [ ]:
no_label.to_csv("no_label_data.csv")

In [ ]:
# xóa toàn bộ dữ liệu chứa rỗng
df = df.dropna()

In [ ]:
# xóa toàn bộ dữ liệu bị lặp
df = df.drop_duplicates(keep="first")

In [ ]:
# xóa toàn bộ dữ liệu không thỏa
valid = ["spam", "ham"]
df = df[df["label"].isin(valid)]

In [ ]:
df.isnull().sum()

In [ ]:
# thay nhãn dán thành số tương ứng
df["label"] = df["label"].map({"ham": 0, "spam": 1})

## 2. Data processing
- chuyển về in thường
- tokenize data
- xóa ký tự đặc biệt
- xóa stop-words và dấu câu
- stemming 

In [ ]:
from nltk.corpus import stopwords
import string 
from nltk.stem.porter import PorterStemmer

In [ ]:
def process_text(text):
    # in thường
    text = text.lower()
    
    # xóa phân cách subject ở đầu text
    if(not text.find("subject")):
        index = len("subject")
        text = text[index:]
    
    # xóa dấu câu
    new_text = ""
    for i in text:
        if (i not in string.punctuation):
            new_text += i
    # token
    new_text = nltk.word_tokenize(new_text)
    res = []
    ps = PorterStemmer()
    for i in new_text:
        # bỏ stop-word
        if (i in stopwords.words("english")):
            continue
        elif (i.isalnum()):
            # stemming
            res.append(ps.stem(i))
        
    return " ".join(res)

In [ ]:
process_text("Do you like my loved pancake on the second table")

In [ ]:
df["processed_text"] = df["text"].apply(process_text)

## 3. Exploring data analysis

In [ ]:
plt.pie(df["label"].value_counts(), labels=["ham", "spam"], autopct="%.2f")

##### Data is imbalanced

In [ ]:
nltk.download("punkt")

In [ ]:
# số ký tự trong nội dung mail
df["num_characters"] = df["text"].apply(len)

In [ ]:
# số từ trong nội dung mail
df["num_words"] = df["text"].apply(lambda x: len(nltk.word_tokenize(x)))

In [ ]:
# số câu trong nội dung mail
df["num_sentences"] = df["text"].apply(lambda x: len(nltk.sent_tokenize(x)))

In [ ]:
# số từ quan trọng trong nội dung mail
df["num_sign_words"] = df["processed_text"].apply(lambda x: len(nltk.word_tokenize(x)))

In [ ]:
df.to_csv("processed_data.csv")

In [ ]:
# phân bố số lượng ký tự, từ, câu
num = ["label", "num_characters", "num_words", "num_sentences", "num_sign_words"]
df_num = pd.DataFrame(df, columns=num)

In [ ]:
# phân bố số ký tự trong nội dung
plt.figure(figsize=(6, 8))
sns.histplot(df_num[df["label"] == 0]["num_characters"])
sns.histplot(df_num[df["label"] == 1]["num_characters"], color="red")

In [ ]:
# phân bố số từ trong nội dung
plt.figure(figsize=(6, 8))
sns.histplot(df_num[df["label"] == 0]["num_words"])
sns.histplot(df_num[df["label"] == 1]["num_words"], color="red")

In [ ]:
# phân bố số câu trong nội dung
plt.figure(figsize=(7, 5))
sns.histplot(df_num[df["label"] == 0]["num_sentences"])
sns.histplot(df_num[df["label"] == 1]["num_sentences"], color="red")

In [ ]:
# phân bố số từ quan trọng trong nội dung
plt.figure(figsize=(6, 8))
sns.histplot(df_num[df["label"] == 0]["num_sign_words"])
sns.histplot(df_num[df["label"] == 1]["num_sign_words"], color="red")

In [ ]:
sns.pairplot(df_num, hue="label")

In [ ]:
# xu hướng của label theo số lượng ký tự, từ, câu
sns.heatmap(df_num.corr(), annot=True)

In [ ]:
# spam words list
spam_corpus = []
for msg in df[df["label"] == 1]["processed_text"].tolist():
    for word in msg:
        spam_corpus.append(word)
len(spam_corpus)

In [ ]:
from collections import Counter
sns.barplot(x = pd.DataFrame(Counter(spam_corpus).most_common(30))[0], y = pd.DataFrame(Counter(spam_corpus).most_common(30))[1])
plt.xticks(rotation="vertical")
plt.show()

In [ ]:
# ham words list
ham_corpus = []
for msg in df[df["label"] == 0]["processed_text"].tolist():
    for word in msg:
        ham_corpus.append(word)
len(ham_corpus)

In [ ]:
from collections import Counter
sns.barplot(x = pd.DataFrame(Counter(ham_corpus).most_common(30))[0], y = pd.DataFrame(Counter(ham_corpus).most_common(30))[1])
plt.xticks(rotation="vertical")
plt.show()

## 4. Model Building
### 4.1 Naive Bayes

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
cv = CountVectorizer()
tf = TfidfVectorizer()

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
gnb = GaussianNB()
mnb = MultinomialNB()
bnb = BernoulliNB()

In [ ]:
X = cv.fit_transform(df["processed_text"]).toarray()
X.shape

In [ ]:
Y = df["label"].values
Y.shape

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

In [ ]:
gnb.fit(X_train, Y_train)
Y_pred1 = gnb.predict(X_test)
print(accuracy_score(Y_test, Y_pred1))
print(confusion_matrix(Y_test, Y_pred1))
print(precision_score(Y_test, Y_pred1))

In [ ]:
mnb.fit(X_train, Y_train)
Y_pred2 = mnb.predict(X_test)
print(accuracy_score(Y_test, Y_pred2))
print(confusion_matrix(Y_test, Y_pred2))
print(precision_score(Y_test, Y_pred2))

In [ ]:
bnb.fit(X_train, Y_train)
Y_pred3 = mnb.predict(X_test)
print(accuracy_score(Y_test, Y_pred3))
print(confusion_matrix(Y_test, Y_pred3))
print(precision_score(Y_test, Y_pred3))